In [2]:
import numpy as np
import pandas as pd

# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

print("labeled train {0}, test data{1}, unlabeled data {2}".format(len(train), len(test), len(unlabeled_train)))

labeled train 25000, test data25000, unlabeled data50000


In [9]:
# basic preprocessing
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def preprocess_review(review, remove_stopwords=False):
    # remove markup
    review_text = BeautifulSoup(review).get_text()
    
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    # 5. Return a list of words
    return words

In [10]:
# tokenizer
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def split_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = [preprocess_review(s, remove_stopwords) for s in raw_sentences if len(s) > 0]
    
    return sentences

In [ ]:
# preprocess data
test_sentences = map((lambda x: split_sentences(x, tokenizer)), train['review'].tolist())
unlabeled_sentences = map((lambda x: split_sentences(x, tokenizer)), unlabeled_train['review'].tolist())

sentences = list(test_sentences) + list(unlabeled_sentences)
print(len(sentences))

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

from gensim.models import word2vec
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)